##### Copyright 2018 The TensorFlow Hub Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
# Copyright 2018 The TensorFlow Hub Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

# Object Detection


<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/hub/tutorials/object_detection"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/hub/tutorials/object_detection.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/hub/tutorials/object_detection.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/hub/tutorials/object_detection.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
  <td>
    <a href="https://tfhub.dev/s?q=google%2Ffaster_rcnn%2Fopenimages_v4%2Finception_resnet_v2%2F1%20OR%20google%2Ffaster_rcnn%2Fopenimages_v4%2Finception_resnet_v2%2F1"><img src="https://www.tensorflow.org/images/hub_logo_32px.png" />See TF Hub models</a>
  </td>
</table>

This Colab demonstrates use of a TF-Hub module trained to perform object detection.

## Setup


In [ ]:
#@title Imports and function definitions

# For running inference on the TF-Hub module.
import tensorflow as tf

import tensorflow_hub as hub

# For downloading the image.
import matplotlib.pyplot as plt
import tempfile
from six.moves.urllib.request import urlopen
from six import BytesIO

# For drawing onto the image.
import numpy as np
from PIL import Image
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageOps

# For measuring the inference time.
import time

# Print Tensorflow version
print(tf.__version__)

# Check available GPU devices.
print("The following GPU devices are available: %s" % tf.test.gpu_device_name())

2.12.0
The following GPU devices are available: /device:GPU:0


## Example use

### Helper functions for downloading images and for visualization.

Visualization code adapted from [TF object detection API](https://github.com/tensorflow/models/blob/master/research/object_detection/utils/visualization_utils.py) for the simplest required functionality.

In [ ]:
def display_image(image):
  fig = plt.figure(figsize=(20, 15))
  plt.grid(False)
  plt.imshow(image)


def download_and_resize_image(url, new_width=256, new_height=256,
                              display=False):
  _, filename = tempfile.mkstemp(suffix=".jpg")
  response = urlopen(url)
  image_data = response.read()
  image_data = BytesIO(image_data)
  pil_image = Image.open(image_data)
  pil_image = ImageOps.fit(pil_image, (new_width, new_height), Image.LANCZOS)
  pil_image_rgb = pil_image.convert("RGB")
  pil_image_rgb.save(filename, format="JPEG", quality=90)
  print("Image downloaded to %s." % filename)
  if display:
    display_image(pil_image)
  return filename


def draw_bounding_box_on_image(image,
                               ymin,
                               xmin,
                               ymax,
                               xmax,
                               color,
                               font,
                               thickness=4,
                               display_str_list=()):
  """Adds a bounding box to an image."""
  draw = ImageDraw.Draw(image)
  im_width, im_height = image.size
  (left, right, top, bottom) = (xmin * im_width, xmax * im_width,
                                ymin * im_height, ymax * im_height)
  draw.line([(left, top), (left, bottom), (right, bottom), (right, top),
             (left, top)],
            width=thickness,
            fill=color)

  # If the total height of the display strings added to the top of the bounding
  # box exceeds the top of the image, stack the strings below the bounding box
  # instead of above.
  display_str_heights = [font.getbbox(ds)[3] for ds in display_str_list]
  # Each display_str has a top and bottom margin of 0.05x.
  total_display_str_height = (1 + 2 * 0.05) * sum(display_str_heights)

  if top > total_display_str_height:
    text_bottom = top
  else:
    text_bottom = top + total_display_str_height
  # Reverse list and print from bottom to top.
  for display_str in display_str_list[::-1]:
    bbox = font.getbbox(display_str)
    text_width, text_height = bbox[2], bbox[3]
    margin = np.ceil(0.05 * text_height)
    draw.rectangle([(left, text_bottom - text_height - 2 * margin),
                    (left + text_width, text_bottom)],
                   fill=color)
    draw.text((left + margin, text_bottom - text_height - margin),
              display_str,
              fill="black",
              font=font)
    text_bottom -= text_height - 2 * margin


def draw_boxes(image, boxes, class_names, scores, max_boxes=10, min_score=0.1):
  """Overlay labeled boxes on an image with formatted scores and label names."""
  colors = list(ImageColor.colormap.values())

  try:
    font = ImageFont.truetype("/usr/share/fonts/truetype/liberation/LiberationSansNarrow-Regular.ttf",
                              25)
  except IOError:
    print("Font not found, using default font.")
    font = ImageFont.load_default()

  for i in range(min(boxes.shape[0], max_boxes)):
    if scores[i] >= min_score:
      ymin, xmin, ymax, xmax = tuple(boxes[i])
      display_str = "{}: {}%".format(class_names[i].decode("ascii"),
                                     int(100 * scores[i]))
      color = colors[hash(class_names[i]) % len(colors)]
      image_pil = Image.fromarray(np.uint8(image)).convert("RGB")
      draw_bounding_box_on_image(
          image_pil,
          ymin,
          xmin,
          ymax,
          xmax,
          color,
          font,
          display_str_list=[display_str])
      np.copyto(image, np.array(image_pil))
  return image

## Apply module

Load a public image from Open Images v4, save locally, and display.

In [ ]:
# By Heiko Gorski, Source: https://commons.wikimedia.org/wiki/File:Naxos_Taverna.jpg
image_url = "https://upload.wikimedia.org/wikipedia/commons/6/60/Naxos_Taverna.jpg"  #@param
downloaded_image_path = download_and_resize_image(image_url, 1280, 856, True)

Pick an object detection module and apply on the downloaded image. Modules:
* **FasterRCNN+InceptionResNet V2**: high accuracy,
* **ssd+mobilenet V2**: small and fast.

In [ ]:
module_handle = "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1" #@param ["https://tfhub.dev/google/openimages_v4/ssd/mobilenet_v2/1", "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"]

detector = hub.load(module_handle).signatures['default']

In [ ]:
def load_img(img):
  print("path: ",img)
  img = tf.io.read_file(img)
  print("tf.io.read_file: ",img)
  img = tf.image.decode_jpeg(img, channels=3)
  print("tf.image.decode_jpeg: ", img)
  return img

In [ ]:
def run_detector(detector, img):
#   img = load_img(img)
#   print("image: ",img)

  converted_img  = tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...]
  start_time = time.time()
  result = detector(converted_img)
  end_time = time.time()-start_time
  print("Time per frame: ",end_time)
  result = {key:value.numpy() for key,value in result.items()}

  print("Found %d objects." % len(result["detection_scores"]))
  print("Inference time: ", end_time-start_time)

  image_with_boxes = draw_boxes(
      img, result["detection_boxes"],
      result["detection_class_entities"], result["detection_scores"],min_score=0.5)
  return image_with_boxes

#   display_image(image_with_boxes)

In [ ]:
# run_detector(detector, downloaded_image_path)

### More images
Perform inference on some additional images with time tracking.


In [ ]:
import cv2
import time
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()


In [ ]:

# def run_detector(model,frame,im_height,im_width):

#     img = frame.reshape(1,im_height,im_width, 3).astype(np.uint8)
#     st_time = time.time()
#     result = model(img)
#     print("frame Time: ",time.time() - st_time)
#     out_img = draw_bboxes(img, result)
#     return out_img

def save_video_with_bounding_boxes(input_video_path, output_video_path, model):
    # Open the input video
    video_capture = cv2.VideoCapture(input_video_path)

    # Get the video's frame width, height, and frames per second
    frame_width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(video_capture.get(cv2.CAP_PROP_FPS))

    # Create VideoWriter object to save the output video
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    output_video = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

    while True:
        # Read the next frame
        ret, frame = video_capture.read()

        if not ret:
            break

        # Perform object detection on the frame
        # Replace the following line with your own object detection code
        # The model variable should contain the logic to detect objects and draw bounding boxes
        # print(frame)
        # frame = tf.convert_to_tensor(frame, dtype=tf.int8)
        # print("converted_frame")
        (im_height, im_width, _)=frame.shape
        # print(type(frame))
        # frame = frame.reshape(1,im_height,im_width, 3).astype(np.uint8)
        # print(frame.shape)
        # return
        # detected_frame =run_detector(model, frame,im_height,im_width)
        detected_frame =run_detector(model, frame)

        # print(detected_frame.shape)
        # image1copy = np.uint8(detected_frame)
        # image1=np.reshape(detected_frame,(im_height,im_width,3))
        # print(image1copy.shape)

        # Write the frame with bounding boxes to the output video
        # output_video.write(detected_frame1)
        output_video.write(detected_frame)
        # Display the frame with bounding boxes
        # cv2.imshow('Video with Bounding Boxes', detected_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the video capture and output video objects
    video_capture.release()
    output_video.release()

    # Close all OpenCV windows
    cv2.destroyAllWindows()

In [ ]:
# Example usage
input_video_path = '/content/4.mp4'
output_video_path = '/content/frcnn/4_frcnn600_50p.mp4'
model = detector  # Replace with your own object detection model

save_video_with_bounding_boxes(input_video_path, output_video_path, model)

Time per frame:  1.3230278491973877
Found 100 objects.
Inference time:  -1692962537.75661
Time per frame:  1.2341349124908447
Found 100 objects.
Inference time:  -1692962539.2093267
Time per frame:  1.2070255279541016
Found 100 objects.
Inference time:  -1692962540.5128882
Time per frame:  1.2273433208465576
Found 100 objects.
Inference time:  -1692962541.740486
Time per frame:  1.3743293285369873
Found 100 objects.
Inference time:  -1692962542.878096
Time per frame:  1.3805367946624756
Found 100 objects.
Inference time:  -1692962544.3401382
Time per frame:  1.2388942241668701
Found 100 objects.
Inference time:  -1692962545.9083695
Time per frame:  1.234250783920288
Found 100 objects.
Inference time:  -1692962547.1960638
Time per frame:  1.2341105937957764
Found 100 objects.
Inference time:  -1692962548.4711592
Time per frame:  1.2294285297393799
Found 100 objects.
Inference time:  -1692962549.7554166
Time per frame:  1.2377352714538574
Found 100 objects.
Inference time:  -1692962551.0